## Tensorflow Low Level API

## Tensors and Operations

In [0]:
tensor=tf.constant([[1,2,3],[4,5,6]])

In [5]:
tensor # kinda like a numpy array

<tf.Tensor: id=0, shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [8]:
tensor[:,0]

<tf.Tensor: id=12, shape=(2,), dtype=int32, numpy=array([1, 4], dtype=int32)>

In [9]:
tensor+1

<tf.Tensor: id=14, shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [5, 6, 7]], dtype=int32)>

In [0]:
import numpy as np
a=np.array([[1,2,3],[2,3,4]])

In [0]:
some_tensor=tf.constant(a) # converting numpy array to tensor

In [0]:
b=some_tensor.numpy() # converting tensor to numpy array

In [15]:
np.max(some_tensor) # numpy operations work with tensorflow tensors

4

Tensorflow doesn't have any implicit type conversion even from int64 to int32

In [0]:
a=tf.constant([1,2,3,4],dtype=tf.float64)

In [18]:
a

<tf.Tensor: id=16, shape=(4,), dtype=float64, numpy=array([1., 2., 3., 4.])>

Use `tf.cast` for type conversion.

In [19]:
a=tf.cast(a,tf.int64)
a

<tf.Tensor: id=17, shape=(4,), dtype=int64, numpy=array([1, 2, 3, 4])>

Tensorflow tensors are immutable.
If your tensor values have to keep changing( like in model weight updation ) use a `tf.Variable` instead.

In [0]:
a=tf.Variable([[1,2,3],[2,3,4]])

In [34]:
a

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [2, 3, 4]], dtype=int32)>

In [35]:
a[0,0].assign(42) # inplace value substitution

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[42,  2,  3],
       [ 2,  3,  4]], dtype=int32)>

## Other Data Structures
* Sparse Tensors : tf.SparseTensor
  * Efficient representation on tensors that are mostly 0s.
* Tensor Arrays : tf.TensorArray
  * List of tensors with same shape and data type.
* Ragged Tensors: tf.RaggedTensor
  * Static list of list of tensors.
* String Tensors:
  * Byte string tensors.
* Sets
* Queues

## Custom Loss Functions

In [0]:
def huber_fn(y_true,_y_pred):
  error=y_true-y_pred
  is_small_error=tf.abs(error) < 1 # if the absolute error is less than one
  squared_loss = tf.square(error)
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error,squared_loss,linear_loss)
#model.compile(loss=huber_fn,optimizer='nadam')
#saving models with custom components
model=keras.models.load_model('some_model.h5',custom_objects={'huber_fn':huber_fn})


## Calculating gradients with AutoDiff

In [0]:
def f(w1,w2):
  return 3*w1**2+2*w1*w2

# the partial derivative of this function with respect to w1 is 6*w1+2*w2
# the partial derivative with respect to w2 is 2*w1

In [0]:
w1,w2=tf.Variable(5.),tf.Variable(3.)
with tf.GradientTape() as tape: # start 'monitoring' operations and storing on a tape
  z=f(w1,w2)


In [57]:
gradients=tape.gradient(z,[w1,w2]) # taking the partial derivatives for functions performed on z and passing the inputs [w1,w2]
#into those functions
gradients

[<tf.Tensor: id=360, shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: id=352, shape=(), dtype=float32, numpy=10.0>]

Here as expected 2 x 5 = 10 and 6 x 5 + 2 x 3 = 36
  * To call gradient more than once you'll have to set tf.GradientTape(persistent=True)

## Tensorflow Graph Function

In [0]:
a=tf.Variable(3.)

In [0]:
def cuber(x):
  return x**3

In [61]:
cuber(a)

<tf.Tensor: id=370, shape=(), dtype=float32, numpy=27.0>

In [64]:
tf_cube=tf.function(cuber) # creates a computational graph and prunes unused nodes to optimize the operation
tf_cube(a)

# or

@tf.function
def tf_cuber(x):
  return x**3
tf_cuber(a)

<tf.Tensor: id=397, shape=(), dtype=float32, numpy=27.0>